In [6]:
!pip install websocket

     |████████████████████████████████| 195 kB 824 kB/s eta 0:00:01
  Created wheel for websocket: filename=websocket-0.2.1-py3-none-any.whl size=192132 sha256=1acf1fd832e2e829f4539ef7e64028bb4f8b9dfd39a3a499cc4dd08d684c9d68
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/cd/8c/c5/42beaa658f4825f4dc80634c34c5a4bb564cdd75545346fa93
Successfully built websocket


In [27]:
!pip uninstall websocket-client

Found existing installation: websocket-client 1.4.1
Uninstalling websocket-client-1.4.1:
  Would remove:
    /Users/mac/opt/anaconda3/bin/wsdump
    /Users/mac/opt/anaconda3/lib/python3.9/site-packages/websocket/*
    /Users/mac/opt/anaconda3/lib/python3.9/site-packages/websocket_client-1.4.1.dist-info/*
  Would not remove (might be manually added):
    /Users/mac/opt/anaconda3/lib/python3.9/site-packages/websocket/data/WebSocketMain.swf
    /Users/mac/opt/anaconda3/lib/python3.9/site-packages/websocket/data/__init__.py
    /Users/mac/opt/anaconda3/lib/python3.9/site-packages/websocket/data/flashsocket.js
    /Users/mac/opt/anaconda3/lib/python3.9/site-packages/websocket/policyserver.py
    /Users/mac/opt/anaconda3/lib/python3.9/site-packages/websocket/server.py
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [28]:
!pip3 install websocket-client

In [5]:
# -*- encoding:utf-8 -*-
import hashlib
import hmac
import base64
from socket import *
import json, time, threading
import websocket
from websocket import create_connection
from urllib.parse import quote
import logging

# reload(sys)
# sys.setdefaultencoding("utf8")
class Client():
    def __init__(self):
        base_url = "ws://rtasr.xfyun.cn/v1/ws"
        ts = str(int(time.time()))
        tt = (app_id + ts).encode('utf-8')
        md5 = hashlib.md5()
        md5.update(tt)
        baseString = md5.hexdigest()
        baseString = bytes(baseString, encoding='utf-8')

        apiKey = api_key.encode('utf-8')
        signa = hmac.new(apiKey, baseString, hashlib.sha1).digest()
        signa = base64.b64encode(signa)
        signa = str(signa, 'utf-8')
        self.end_tag = "{\"end\": true}"

        self.ws = create_connection(base_url + "?appid=" + app_id + "&ts=" + ts + "&signa=" + quote(signa))
        self.trecv = threading.Thread(target=self.recv)
        self.trecv.start()

    def send(self, file_path):
        file_object = open(file_path, 'rb')
        try:
            index = 1
            while True:
                chunk = file_object.read(1280)
                if not chunk:
                    break
                self.ws.send(chunk)

                index += 1
                time.sleep(0.04)
        finally:
            file_object.close()

        self.ws.send(bytes(self.end_tag.encode('utf-8')))
        print("send end tag success")

    def recv(self):
        try:
            while self.ws.connected:
                result = str(self.ws.recv())
                if len(result) == 0:
                    print("receive result end")
                    break
                result_dict = json.loads(result)
                # 解析结果
                if result_dict["action"] == "started":
                    print("handshake success, result: " + result)

                if result_dict["action"] == "result":
                    result_1 = result_dict
                    # result_2 = json.loads(result_1["cn"])
                    # result_3 = json.loads(result_2["st"])
                    # result_4 = json.loads(result_3["rt"])
                    print("rtasr result: " + result_1["data"])

                if result_dict["action"] == "error":
                    print("rtasr error: " + result)
                    self.ws.close()
                    return
        except websocket.WebSocketConnectionClosedException:
            print("receive result end")

    def close(self):
        self.ws.close()
        print("connection closed")


In [7]:
if __name__ == '__main__':
    logging.basicConfig()

    app_id = "fb41b9d3"
    api_key = "76fe5ecced751ed5f9395d0c5a39e92b"
    file_path = r"./test_1.pcm"

    client = Client()
    client.send(file_path)



handshake success, result: {"action":"started","code":"0","data":"","desc":"success","sid":"rta008d07f3@dx1e8a169cd81f000100"}
rtasr result: {"seg_id":0,"cn":{"st":{"rt":[{"ws":[{"cw":[{"sc":0.00,"w":"床","wp":"n","rl":"0","wb":10,"wc":0.00,"we":20}],"wb":0,"we":0},{"cw":[{"sc":0.00,"w":"前","wp":"n","rl":"0","wb":20,"wc":0.00,"we":30}],"wb":0,"we":0}]}],"bg":"0","type":"1","ed":"0"}},"ls":false}
rtasr result: {"seg_id":1,"cn":{"st":{"rt":[{"ws":[{"cw":[{"sc":0.00,"w":"床","wp":"n","rl":"0","wb":13,"wc":0.00,"we":26}],"wb":0,"we":0},{"cw":[{"sc":0.00,"w":"前","wp":"n","rl":"0","wb":26,"wc":0.00,"we":39}],"wb":0,"we":0},{"cw":[{"sc":0.00,"w":"明","wp":"n","rl":"0","wb":39,"wc":0.00,"we":52}],"wb":0,"we":0},{"cw":[{"sc":0.00,"w":"月光","wp":"n","rl":"0","wb":52,"wc":0.00,"we":65}],"wb":0,"we":0}]}],"bg":"130","type":"1","ed":"0"}},"ls":false}
rtasr result: {"seg_id":2,"cn":{"st":{"rt":[{"ws":[{"cw":[{"sc":0.00,"w":"床","wp":"n","rl":"0","wb":22,"wc":0.00,"we":44}],"wb":0,"we":0},{"cw":[{"sc":0.0

rtasr result: {"seg_id":9,"cn":{"st":{"rt":[{"ws":[{"cw":[{"sc":0.00,"w":"床","wp":"n","rl":"0","wb":1,"wc":0.00,"we":20}],"wb":1,"we":20},{"cw":[{"sc":0.00,"w":"前","wp":"n","rl":"0","wb":21,"wc":0.00,"we":40}],"wb":21,"we":40},{"cw":[{"sc":0.00,"w":"明","wp":"n","rl":"0","wb":41,"wc":0.00,"we":64}],"wb":41,"we":64},{"cw":[{"sc":0.00,"w":"月光","wp":"n","rl":"0","wb":65,"wc":0.00,"we":116}],"wb":65,"we":116},{"cw":[{"sc":0.00,"w":"，","wp":"p","rl":"0","wb":121,"wc":0.00,"we":121}],"wb":121,"we":121},{"cw":[{"sc":0.00,"w":"疑","wp":"n","rl":"0","wb":121,"wc":0.00,"we":148}],"wb":121,"we":148},{"cw":[{"sc":0.00,"w":"是","wp":"n","rl":"0","wb":149,"wc":0.00,"we":164}],"wb":149,"we":164},{"cw":[{"sc":0.00,"w":"地上","wp":"n","rl":"0","wb":165,"wc":0.00,"we":208}],"wb":165,"we":208},{"cw":[{"sc":0.00,"w":"霜","wp":"n","rl":"0","wb":209,"wc":0.00,"we":244}],"wb":209,"we":244},{"cw":[{"sc":0.00,"w":"，","wp":"p","rl":"0","wb":249,"wc":0.00,"we":249}],"wb":249,"we":249},{"cw":[{"sc":0.00,"w":"举","wp":"n

In [9]:
if __name__ == '__main__':
    logging.basicConfig()

    app_id = "fb41b9d3"
    api_key = "76fe5ecced751ed5f9395d0c5a39e92b"
    file_path = r"./test_2.wav"

    client = Client()
    client.send(file_path)


handshake success, result: {"action":"started","code":"0","data":"","desc":"success","sid":"rta008d14bf@dx1e8a169cecec000100"}
send end tag success
rtasr result: {"seg_id":0,"cn":{"st":{"rt":[{"ws":[{"cw":[{"sc":0.00,"w":"","wp":"n","rl":"0","wb":0,"wc":0.00,"we":0}],"wb":0,"we":0}]}],"bg":"0","type":"0","ed":"0"}},"ls":true}
receive result end
